In [1]:
import pandas as pd
import numpy as np
import time
import pickle

In [3]:
pid = pd.read_csv('../data/processed/pitchers/pitcher_ids.csv')

In [3]:
pid.head()

,pitches,player_id,player_name,total_pitches,pitch_percent,ba,iso,babip,slg,woba,...,takes,eff_min_vel,release_extension,pos3_int_start_distance,pos4_int_start_distance,pos5_int_start_distance,pos6_int_start_distance,pos7_int_start_distance,pos8_int_start_distance,pos9_int_start_distance
0,3687,545333,Trevor Bauer,3687,100.0,0.230,0.199,0.290,0.429,0.320,...,1999,-0.4,6.00,111,153,115,148,291,311,289
1,3553,458681,Lance Lynn,3553,100.0,0.243,0.146,0.322,0.390,0.297,...,1803,0.0,6.23,110,154,114,149,293,322,300
2,3495,593958,Eduardo Rodriguez,3495,100.0,0.253,0.138,0.318,0.391,0.310,...,1865,-0.5,5.92,107,150,115,149,287,322,288
3,3448,434378,Justin Verlander,3448,100.0,0.172,0.188,0.219,0.361,0.246,...,1652,-0.9,5.83,113,157,117,152,290,316,297
4,3384,544931,Stephen Strasburg,3384,100.0,0.210,0.139,0.276,0.349,0.268,...,1761,0.8,6.71,114,151,112,149,303,316,296


In [4]:
pid.player_id.unique()[0]

545333

In [5]:
pid.total_pitches.sum()

598053

## Create loop to scrape pitcher data dfs into dictionary

In [6]:
def url_per_pitcher(front_url, pitcher_id, back_url):
    pitcher_url = front_url + str(pitcher_id) + back_url
    pitcher_url = pitcher_url.replace(' ', '')
    return pitcher_url

In [7]:
# Get list of all pither ids
pids = pid['player_id'].tolist()

In [8]:
# Create url variables for looping
# Front part of download url
front_url = 'https://baseballsavant.mlb.com/statcast_search/csv?all=true&hfPT=&hfAB=&hfBBT=&hfPR=&hfZ=&\
stadium=&hfBBL=&hfNewZones=&hfGT=R%7C&hfC=&hfSea=2019%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&\
pitcher_throws=&batter_stands=&hfSA=&game_date_gt=&game_date_lt=&hfInfield=&team=&position=&hfOutfield=&\
hfRO=&home_road=&hfFlag=&hfPull=&pitchers_lookup%5B%5D='

# List of pitcher ids
pids = pid['player_id'].tolist()

# Back part of download url
back_url = '&metric_1=&hfInn=&min_pitches=0&min_results=0&group_by=name&sort_col=pitches&\
player_event_sort=h_launch_speed&sort_order=desc&min_pas=0&chk_pitch_type=on&chk_pitch_result=on&\
chk_count=on&chk_batter_stands=on&chk_inning=on&chk_metric1_gt=on&chk_metric2_gt=on&chk_metric3_gt=on&\
chk_outs=on&chk_pitcher_throws=on&chk_runner_on=on&chk_metric1_lt=on&chk_metric2_lt=on&chk_metric3_lt=on&\
type=details&'

In [9]:
# Test function to see if it works and it does
url_per_pitcher(front_url, pids[0], back_url);

In [10]:
# Create dictionary of pitcher names and baseaball savant urls
pitcher_urls_dict = {}

for p in pids:
    pitcher_url = url_per_pitcher(front_url, p, back_url)
    pitcher_urls_dict.update({str(p): pitcher_url})

In [11]:
pitcher_urls_dict;

In [12]:
# Create a function that takes a pitcher id and url
# It captues csv batter data for that pitcher
# Returns a df

def get_pitcher_data(p_id, url):
    pitches_df = pd.read_csv(url)
    name = pitches_df.player_name.unique()[0]
    return str(p_id), pitches_df

In [13]:
# Create dictionary with which to save all pitcher dataframes acquired from baseball savant
pitcher_df_dict = {}
total_pitchers = len(pids)
pitchers_left = len(pids) - 1

for p in pitcher_urls_dict:  
    p_id, pitcher_df = get_pitcher_data(p, pitcher_urls_dict[p])
    pitcher_df_dict.update({p_id: pitcher_df})
    print('Just snagged pitcher data for {}: {} of {} snags left...'.format(p_id, pitchers_left, total_pitchers))
    pitchers_left -= 1
    time.sleep(20)
    
# After snagging data for half the league (15 teams) hit a 502 error
# Disabling code so webscraping does not rerun

Just snagged pitcher data for 545333: 372 of 373 snags left...
Just snagged pitcher data for 458681: 371 of 373 snags left...
Just snagged pitcher data for 593958: 370 of 373 snags left...
Just snagged pitcher data for 434378: 369 of 373 snags left...
Just snagged pitcher data for 544931: 368 of 373 snags left...
Just snagged pitcher data for 543037: 367 of 373 snags left...
Just snagged pitcher data for 501985: 366 of 373 snags left...
Just snagged pitcher data for 669456: 365 of 373 snags left...
Just snagged pitcher data for 605400: 364 of 373 snags left...
Just snagged pitcher data for 571578: 363 of 373 snags left...
Just snagged pitcher data for 594798: 362 of 373 snags left...
Just snagged pitcher data for 518516: 361 of 373 snags left...
Just snagged pitcher data for 594835: 360 of 373 snags left...
Just snagged pitcher data for 656427: 359 of 373 snags left...
Just snagged pitcher data for 625643: 358 of 373 snags left...
Just snagged pitcher data for 554430: 357 of 373 snags 

Just snagged pitcher data for 605446: 241 of 373 snags left...
Just snagged pitcher data for 601713: 240 of 373 snags left...
Just snagged pitcher data for 642547: 239 of 373 snags left...
Just snagged pitcher data for 641312: 238 of 373 snags left...
Just snagged pitcher data for 663567: 237 of 373 snags left...
Just snagged pitcher data for 543557: 236 of 373 snags left...
Just snagged pitcher data for 570240: 235 of 373 snags left...
Just snagged pitcher data for 656288: 234 of 373 snags left...
Just snagged pitcher data for 643327: 233 of 373 snags left...
Just snagged pitcher data for 521655: 232 of 373 snags left...
Just snagged pitcher data for 664199: 231 of 373 snags left...
Just snagged pitcher data for 502748: 230 of 373 snags left...
Just snagged pitcher data for 461829: 229 of 373 snags left...
Just snagged pitcher data for 642545: 228 of 373 snags left...
Just snagged pitcher data for 502706: 227 of 373 snags left...
Just snagged pitcher data for 660761: 226 of 373 snags 

Just snagged pitcher data for 656685: 110 of 373 snags left...
Just snagged pitcher data for 489446: 109 of 373 snags left...
Just snagged pitcher data for 592135: 108 of 373 snags left...
Just snagged pitcher data for 453284: 107 of 373 snags left...
Just snagged pitcher data for 592229: 106 of 373 snags left...
Just snagged pitcher data for 593140: 105 of 373 snags left...
Just snagged pitcher data for 605155: 104 of 373 snags left...
Just snagged pitcher data for 458006: 103 of 373 snags left...
Just snagged pitcher data for 518774: 102 of 373 snags left...
Just snagged pitcher data for 554340: 101 of 373 snags left...
Just snagged pitcher data for 542960: 100 of 373 snags left...
Just snagged pitcher data for 489265: 99 of 373 snags left...
Just snagged pitcher data for 444468: 98 of 373 snags left...
Just snagged pitcher data for 547973: 97 of 373 snags left...
Just snagged pitcher data for 642152: 96 of 373 snags left...
Just snagged pitcher data for 501625: 95 of 373 snags left.

In [14]:
# New loop to grab pitchers not acquired prior to the 502 error thrown above

for pitcher in pitcher_urls_dict:
    if pitcher in pitcher_df_dict:
        print('Skipping {}, already grabbed it'.format(pitcher))
    else:
        p_id, pitcher_df = get_pitcher_data(p, pitcher_urls_dict[p])
        pitcher_df_dict.update({p_id: pitcher_df})
        print('Just snagged pitcher data for {}'.format(pitcher))
        time.sleep(20)

Skipping 545333, already grabbed it
Skipping 458681, already grabbed it
Skipping 593958, already grabbed it
Skipping 434378, already grabbed it
Skipping 544931, already grabbed it
Skipping 543037, already grabbed it
Skipping 501985, already grabbed it
Skipping 669456, already grabbed it
Skipping 605400, already grabbed it
Skipping 571578, already grabbed it
Skipping 594798, already grabbed it
Skipping 518516, already grabbed it
Skipping 594835, already grabbed it
Skipping 656427, already grabbed it
Skipping 625643, already grabbed it
Skipping 554430, already grabbed it
Skipping 622491, already grabbed it
Skipping 450203, already grabbed it
Skipping 621244, already grabbed it
Skipping 571510, already grabbed it
Skipping 425844, already grabbed it
Skipping 592789, already grabbed it
Skipping 645261, already grabbed it
Skipping 592662, already grabbed it
Skipping 573186, already grabbed it
Skipping 452657, already grabbed it
Skipping 527054, already grabbed it
Skipping 502188, already gra

In [15]:
#Test df dict
pitcher_df_dict['543883']

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,FF,2019-09-29,89.8,-2.8772,5.3142,Nick Vincent,621573,543883,field_out,hit_into_play,...,1,4,4,1,4,1,4,1,Standard,Standard
1,FF,2019-09-29,89.5,-2.9731,5.3260,Nick Vincent,621573,543883,NaN,foul,...,1,4,4,1,4,1,4,1,Standard,Standard
2,FF,2019-09-29,89.9,-3.0888,5.2808,Nick Vincent,621573,543883,NaN,foul,...,1,4,4,1,4,1,4,1,Standard,Standard
3,FF,2019-09-29,88.1,-2.9872,5.5094,Nick Vincent,621573,543883,NaN,called_strike,...,1,4,4,1,4,1,4,1,Standard,Standard
4,FC,2019-09-29,87.3,-2.8155,5.5738,Nick Vincent,621573,543883,NaN,ball,...,1,4,4,1,4,1,4,1,Standard,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,FF,2019-03-28,90.2,-2.8610,5.5170,Nick Vincent,614173,543883,NaN,ball,...,2,0,2,0,0,2,2,0,Infield shift,Standard
747,FC,2019-03-28,86.8,-2.6194,5.7730,Nick Vincent,614173,543883,NaN,foul,...,2,0,2,0,0,2,2,0,Infield shift,Standard
748,FC,2019-03-28,86.4,-2.6513,5.8020,Nick Vincent,614173,543883,NaN,swinging_strike,...,2,0,2,0,0,2,2,0,Infield shift,Standard
749,FT,2019-03-28,89.6,-2.4109,5.7907,Nick Vincent,595978,543883,single,hit_into_play_no_out,...,2,0,2,0,0,2,2,0,Strategic,Standard


In [16]:
# Loop through batter_df_dict and concatitate all dfs into one master fil
for pitcher in pitcher_df_dict.keys():
    if pitcher == '545333':
        all_pitcher_df = pitcher_df_dict[pitcher]
    else:
        all_pitcher_df = pd.concat([all_pitcher_df, pitcher_df_dict[pitcher]])

In [17]:
all_pitcher_df

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,FF,2019-09-22,93.2,-1.5424,5.4118,Trevor Bauer,643446,545333,field_out,hit_into_play,...,2,5,5,2,5,2,5,2,Infield shift,Standard
1,FC,2019-09-22,85.6,-1.5705,5.4798,Trevor Bauer,643446,545333,NaN,swinging_strike,...,2,5,5,2,5,2,5,2,Infield shift,Standard
2,FC,2019-09-22,85.9,-1.5974,5.4080,Trevor Bauer,643446,545333,NaN,ball,...,2,5,5,2,5,2,5,2,Infield shift,Standard
3,KC,2019-09-22,78.4,-1.4179,5.7155,Trevor Bauer,607043,545333,field_out,hit_into_play,...,2,5,5,2,5,2,5,2,Infield shift,Standard
4,FF,2019-09-22,94.0,-1.4797,5.3880,Trevor Bauer,607043,545333,NaN,foul,...,2,5,5,2,5,2,5,2,Infield shift,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,FF,2019-03-28,90.2,-2.8610,5.5170,Nick Vincent,614173,543883,NaN,ball,...,2,0,2,0,0,2,2,0,Infield shift,Standard
747,FC,2019-03-28,86.8,-2.6194,5.7730,Nick Vincent,614173,543883,NaN,foul,...,2,0,2,0,0,2,2,0,Infield shift,Standard
748,FC,2019-03-28,86.4,-2.6513,5.8020,Nick Vincent,614173,543883,NaN,swinging_strike,...,2,0,2,0,0,2,2,0,Infield shift,Standard
749,FT,2019-03-28,89.6,-2.4109,5.7907,Nick Vincent,595978,543883,single,hit_into_play_no_out,...,2,0,2,0,0,2,2,0,Strategic,Standard


In [18]:
outfile = open('../../../Capstone/no_git/all_pitcher_data_2019.pickle','wb')
pickle.dump(all_pitcher_df ,outfile)
outfile.close()